# Visualize the Board Area Controlled by Shipyards With Voronoi Diagrams

This notebook shows a simple implementation of [Voronoi diagrams](https://en.wikipedia.org/wiki/Voronoi_diagram) to determine the region (board grid cells) that are nearest to each player's shipyard. 

This visualization easily shows which areas of the board are "controlled territory" of each player.

This can then be used to implement rules specific to shipyards that are near the boundary (more likely to be attached? better positioned to make a quick strike on other enemy boundary shipyards?)
Or rules specific to shipyards that are far from the boundary -> less risk of attack so can focus on mining / shipbuilding?

For analyizing matches, this visualization is also useful to get an immediate understanding of the evolution of the controlled area by each player across turns. And get easily see the 'contested areas' with shipyard combat.

Note: the calculation of the Voronoi diagram itself takes about 200ms (on Kaggle notebook) but could be easily paralellized (independent for loops)


In [ ]:
###
### Voronoi classes adapted (simplified) from @xiaoxiae https://github.com/xiaoxiae/Voronoi/blob/master/voronoi.py
###

from random import randint
from typing import *
from math import hypot, sqrt
import numpy as np

class RegionAlgorithm:
    def randomized(width: int, height: int, regions: int) -> List[Tuple[int, int]]:
        """Return regions that are entirely random."""
        points = []
        while len(points) != regions:
            p = (randint(0, width - 1), randint(0, height - 1))
            if p in points:
                continue
            points.append(p)
        return points

    def uniform(width: int, height: int, regions: int) -> List[Tuple[int, int]]:
        """Return regions that attempt to be somewhat uniform."""
        k = 10
        points = []
        while len(points) != regions:
            best_p = None
            d_max = 0
            for _ in range(k * len(points) + 1):
                p = (randint(0, width - 1), randint(0, height - 1))
                if p in points:
                    continue
                if len(points) == 0:
                    best_p = p
                    break
                d_min = float('inf')
                for x, y in points:
                    d = hypot(p[0]-x, p[1]-y)
                    if d < d_min:
                        d_min = d
                if d_min > d_max:
                    d_max = d_min
                    best_p = p
            if best_p is None:
                continue
            points.append(best_p)
        return points


class DistanceAlgorithm:
    def euclidean(x, y, xn, yn):
        """Calculate the image regions (up to a distance) using euclidean distance."""
        return hypot(xn-x, yn-y)

    def manhattan(x, y, xn, yn):
        """Calculate the image regions using manhattan distance."""
        return abs(xn-x) + abs(yn-y)

    def euclidean45degrees(x, y, xn, yn):
        """Calculate the image regions using euclidean, but allow only lines in 45 degree increments."""
        return sqrt(2 * min(abs(xn-x), abs(yn-y)) ** 2) + abs(abs(xn-x) - abs(yn-y))

    def chebyshev(x, y, xn, yn):
        """Calculate the image regions using chebyshev distance."""
        return min(abs(xn-x), abs(yn-y)) + abs(abs(xn-x) - abs(yn-y))

    def set_each_point(
        width: int, height: int,
        region_centers: List[Tuple[int, int]],
        image: np.ndarray,
        distance_function
    ):
        """Calculate the image regions using the provided metric."""

        for x in range(width):
            for y in range(height):
                d_min = float('inf')
                for i, region in enumerate(region_centers):
                    xn, yn = region
                    d = distance_function(x, y, xn, yn)
                    if d < d_min:
                        d_min = d
                        image[x, y] = i

def generate_voronoi(
        regions: int,
        width: int = 1920,
        height: int = 1080,
        region_algorithm = RegionAlgorithm.uniform,
        distance_algorithm = DistanceAlgorithm.manhattan,
        wrap_around=False,
        return_region=True
):
    """Generate a voronoi diagram of size width x height using the distance_algorithm
    if regions is an int, n=regions region centers will be assigned according to the region_algorithm
    otherwise, regions should be a list of tuples (indexes of region centers)
    if wrap_around is True, region centers are duplicated above/below/left/right (9 copies) and width and height multiplied by 3

    returns an array representing the voronoi diagrams. each value is a unique region id (incrementing with each region center)
    """

    if type(regions) == list:
        # format: list of tuple coordinates. Origin is bottom left of array (!)
        region_centers = regions
    else:
        # create a list of n=regions region centers
        region_centers = region_algorithm(width, height, regions)

    if wrap_around:
        # duplicate region centeres above/below/left/right (9 copies)
        # to emulate the game board wrapping around on 4 sides
        region_centers_wrapped = []
        for center in region_centers:
            for i in range(3):
                for j in range(3):
                    region_centers_wrapped.append(
                        [center[0] + i * height, center[1] + j * width]
                    )
        region_centers = region_centers_wrapped
        height *= 3
        width *= 3

    # output image with all regions assigned their voronoi diagram ids
    image = np.zeros((height, width))
    DistanceAlgorithm.set_each_point(width, height, region_centers, image, distance_algorithm)

    if return_region:
        # image containing the region centers
        region_image = np.zeros((height, width))
        for i, r in enumerate(region_centers):
            region_image[r[0], r[1]] = i
        return image, region_image
    else:
        return image

## Sample usage to generate Voronoi diagrams easily

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

In [ ]:
%%time
image_voronoi = generate_voronoi(
    width = 21,
    height = 21,
    regions = 40,
    return_region=False,
    distance_algorithm = DistanceAlgorithm.manhattan
)

In [ ]:
px.imshow(image_voronoi, text_auto=True)

## Sample usage to generate a Voronoi diagram with input = a list of shipyards coordinates, and output = array ids with two classes (each player)

note that this implementation does indeed take into account that the board wraps around the top/bottom/left/right!

In [ ]:
shipyard_positions = [
    (5, 15),
    (10, 15),
    (6, 13),
    (6, 10),
    (5, 18),
    (2, 18),
    (2, 0),
    (6, 20),
    (15, 5),
    (10, 5),
    (14, 7),
    (14, 10),
    (13, 12),
    (15, 2),
    (17, 7),
    (19, 10)
]
board_size = 21

In [ ]:
%%time
# generate voronoi diagrams using the default manhattan / taxicab distance (which is what makes sense for flight plans in Kore2022)
image = generate_voronoi(
    width=board_size,
    height=board_size,
    regions=shipyard_positions,
    wrap_around=True,
    return_region=False,
    distance_algorithm = DistanceAlgorithm.manhattan
)
# unwrap image (center crop)
image = image[board_size:-board_size, board_size:-board_size]
# assign voronoi ids to player 0 / player 1
wrap_factor = 9
n_shipyards_p0 = 8
image = (image  > n_shipyards_p0 * wrap_factor).astype('int')
# re-orient images to match plotly convention -> origin on top left instead of bottom left
image = np.rot90(image)

In [ ]:
px.imshow(image, text_auto=True, color_continuous_scale=['blue', 'red'])

## Sample usage to visualize the evolution of controlled areas on the board during an entire match

#### Option 1 run a match local 
e.g your baseline

In [ ]:
from kaggle_environments import make
env = make("kore_fleets", debug=True)
env.run(["balanced", "balanced"])
env.render(mode="ipython", width=1000, height=800)

#### Option 2 get a match from Kaggle Leaderboard

e.g today's top is 1Musketeer

In [ ]:
import json, requests
import kaggle_environments
# from @huikang at https://www.kaggle.com/code/huikang/kore-2022-match-analysis/notebook?scriptVersionId=98112789
def fix_overage_time(match):
    for turn_idx, match_state in enumerate(match["steps"]):
        for player_id in [0,1]:
            match_state[player_id]["observation"]["remainingOverageTime"] \
                = max(0, match_state[player_id]["observation"]["remainingOverageTime"])
    return match

def load_from_episode_id(episode_id):
    base_url = "https://www.kaggle.com/api/i/competitions.EpisodeService/"
    get_url = base_url + "GetEpisodeReplay"
    req = requests.post(get_url, json = {"episodeId": int(episode_id)}).json()
    match = json.loads(req["replay"])
    match = fix_overage_time(match)
    env = kaggle_environments.make(
        "kore_fleets", 
        steps=match['steps'],
        configuration=match['configuration'],
    )
    return env
env = load_from_episode_id(38902186)  # e.g today's top is 1Musketeer
env.render(mode="ipython", width=1000, height=800)

Parse the game results and calculate Voronoi diagrams for each steps 

In [ ]:
SIZE = 21

def _get_col_row(size, pos):
    return pos % size, (size - 1) - (pos // size)

def parse_shipyards(envsteps):
    return [
        [step[0]['observation']['players'][playerid][1] for step in envsteps] 
        for playerid in range(2) # player 0 and 1
    ]

def calculate_voronoi(step, shipyards, process=False):
    image, shipyard_image = generate_voronoi(
        width=SIZE,
        height=SIZE,
        regions=[_get_col_row(SIZE, pos=sy[0]) for playerid in range(2) for sy in shipyards[playerid][step].values()],
        wrap_around=True,
        distance_algorithm = DistanceAlgorithm.manhattan
    )
    n_shipyards_p0 = len(shipyards[0][step])
    wrap_factor = 9
    image = (image  > n_shipyards_p0 * wrap_factor).astype('int')
    # re-orient images to match plotly convention -> origin on top left instead of bottom left
    image = np.rot90(image)
    shipyard_image = np.rot90(shipyard_image)
    if process:
        return process_voronoi(image, shipyard_image, n_shipyards_p0 * wrap_factor)
    else:
        return image, shipyard_image

def process_voronoi(image, shipyard_image, sy_cutoff):
    # optional additional post-processing for visualization
    # add the position of each players shipyards to the the voronoi image
    together = image[SIZE:-SIZE, SIZE:-SIZE].astype('float')
    shipyard_image = shipyard_image[SIZE:-SIZE, SIZE:-SIZE]
    together[np.where(shipyard_image > sy_cutoff)] = 1.5  # player 1 shipyards will be bright red
    together[np.where((shipyard_image < sy_cutoff) & (shipyard_image > 0))] = -0.5  # player 0 shipyards will be bright blue
    return together

In [ ]:
shipyards = parse_shipyards(env.steps)
N = len(env.steps)
cached_voronois = [
    calculate_voronoi(step, shipyards, process=True) 
        for step in range(N)
]

In [ ]:
# visualize for any given step
px.imshow(cached_voronois[-50], color_continuous_scale=['blue', 'red'], range_color=[-0.5,1.5])

In [ ]:
# visualize the entire match
fig = go.Figure(
    data=[go.Heatmap(z=cached_voronois[0], colorscale=['blue', 'red'], zmin=-0.5,zmax=1.5)],
    layout=go.Layout(
        height=800,
        width=800,
        yaxis_scaleanchor="x",  # fixed aspect ratio
        yaxis_autorange="reversed",
        plot_bgcolor='rgba(0,0,0,0)',  # transparent background
        updatemenus=[dict(
            type="buttons",
            buttons=[dict(label="Play",
                          method="animate",
                          args=[None,
                                 {"frame": {"duration": 3},
                                "mode": "immediate",
                                "transition": {"duration": 0}}]),
                    dict(label="Pause",
                         method="animate",
                         args=[[None],
                               {"frame": {"duration": 0, "redraw": False},
                                "mode": "immediate",
                                "transition": {"duration": 0}}],
                         )])
            ],
        sliders=[{
            'yanchor': 'bottom', 'xanchor': 'left',
            'currentvalue': {
                'font': {'size': 20},
                'prefix': 'Frame:',
                'visible': True,
            },
            'len': 1.2, 'x': -0.1, 'y': -0.15,
            'steps': [
                {
                    'method': 'animate',
                    'label': step + 1,
                    'args': [[step], {'frame': {'duration': 0, 'redraw': True},
                        'mode': 'immediate'}
                    ],
                } for step in range(N)
            ] 
        }]
    ),
    frames=[
        go.Frame(
            data=[go.Heatmap(z=cached_voronois[step])],
            name=step
        ) 
        for step in range(1, N)       
    ]
)
fig.show()

In this visualization the area controlled by each player during the entire match can be seen intuitively

Frontier areas with shipyard combat can be grasped

Rules involving shipyards on the frontier (more likely to be attacked?) vs shipyards in the core (safer?) can be tested/verified


Notice that the "wrap around the board" is correctly reflected in the control areas. And using the Manhattan distance gives the control areas as a 'flight plan' distance